## Purpose: Create new MIRI imager reference files for CDP7 ##

First import the things that we need from the pipeline code

In [1]:
import os as os
import numpy as np
import pdb as pdb
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri

Import the MIRI coordinates code from https://github.com/STScI-MIRI/coordinates and ensure that it is on the PYTHONPATH.  Also ensure that the output data directory is set:

setenv MIRICOORD_DATA_DIR /YourLocalPathToData/ (this is where io will happen)

In [2]:
data_dir=os.path.expandvars('$MIRICOORD_DATA_DIR')

Import the standalone MIRI imager tools that use the FITS distortion files, and set to CDP-7

In [3]:
import miricoord.imager.mirim_tools as mt
mt.set_toolversion('cdp7')

Import the MIRI imager pipeline tools that can work with the CRDS distortion files, and set to CDP-7

In [4]:
import miricoord.imager.mirim_pipetools as mpt
mpt.set_toolversion('cdp7')

Import the code that actually creates the CRDS distortion files

In [5]:
import miricoord.imager.makecrds.makecrds_ima_cdp7 as cdpcreate

Point to the IDT-delivered CDP file that we're using

In [6]:
cdpfile=mt.get_fitsreffile()

Specify our output filenames for the two asdf files

In [7]:
distfile=os.path.join(data_dir,"temp/jwst_miri_imager_distortion_cdp7.asdf")

In [8]:
offfile=os.path.join(data_dir,"temp/jwst_miri_filteroffset_cdp7.asdf")

Call the code to actually make the reference files

In [9]:
cdpcreate.make_references(cdpfile,{'DISTORTION': distfile, 'FILTEROFFSET':offfile})

Now point to these reference files and load the transform into the function 'xform'

In [10]:
refs = {"distortion": distfile, "filteroffset": offfile}
refs

{'distortion': '/ifs/jwst/wit/miri/gitdata/STScI-MIRI/miricoord/temp/jwst_miri_imager_distortion_cdp7.asdf',
 'filteroffset': '/ifs/jwst/wit/miri/gitdata/STScI-MIRI/miricoord/temp/jwst_miri_filteroffset_cdp7.asdf'}

In [11]:
xform=mpt.xytov2v3model('F770W',refs=refs)

The pipeline uses 0-indexed detector pixels; the MIRI ASDF object assumes this form of input and converts as necessary to the reference frames needed by various IDT transforms.  The reference point of the imager is 692.5, 511.5 in this 0-indexed convention; test that it gives the correct v2,v3 (-453.559, -373.814)

In [12]:
x,y=692.5,511.5

In [13]:
v2,v3=xform(x,y)

In [14]:
print(v2,v3)

-453.5591160661637 -373.81444687292515


And the inverse transform

In [15]:
xform.inverse(-453.559,-373.814)

(692.501834160098, 511.49268215536677)

This location (-453.559, -373.814) which changed in CDP-7 is the agreed-upon reference point. 
Now run a full test of the transform against a table of known values using a dedicated testing script.  If no output, it worked.

In [16]:
cdpcreate.test_transform(refs=refs)

As of Apr 12 2017 we can't test the bounding box implementation directly because it is build into the pipeline, not the model class.  Therefore we must test this by actually running the pipeline on MIRI Imager test and reading the transform in from there.  This is done with the mirim_imagertest.ipynb notebook

In [17]:
xform

<CompoundModel(offset_0=-0., offset_1=-0., offset_2=-4., c0_4=-12.7875, c1_4=0.025, c0_5=-12.7875, c1_5=0.025, c0_0_7=0.00147138, c1_0_7=0.01109653, c2_0_7=-0.00130369, c3_0_7=0.00000003, c4_0_7=0.00000073, c0_1_7=4.4505596, c0_2_7=-0.00020939, c0_3_7=-0.00019305, c0_4_7=0.00000523, c1_1_7=0.00022353, c1_2_7=0.00000126, c1_3_7=0.00000001, c2_1_7=-0.00010173, c2_2_7=0.00000505, c3_1_7=0.00000001, c0_0_8=0.00313941, c1_0_8=4.41486204, c2_0_8=0.00017775, c3_0_8=0.00000381, c4_0_8=0., c0_1_8=-0.0220801, c0_2_8=-0.0000569, c0_3_8=0.00000188, c0_4_8=-0.00000003, c1_1_8=-0.00279708, c1_2_8=-0.00005583, c1_3_8=0.00000338, c2_1_8=0.00000044, c2_2_8=-0.00000002, c3_1_8=0.00000238, c0_0_10=-375.39, c1_0_10=0.99684849, c0_1_10=0.07932904, c0_0_11=-434.08, c1_0_11=0.07932904, c0_1_11=-0.99684849)>